In [1]:
import pandas as pd

In [2]:
patients = pd.read_csv('../data/patients_ibd.csv')
diagnoses = pd.read_csv('../data/diagnoses_ibd.csv')
admissions = pd.read_csv('../data/admissions_ibd.csv')

patients['dod'] = pd.to_datetime(patients['dod'])
patients['anchor_year'] = pd.to_datetime(patients['anchor_year'].apply(lambda x: f'{x}-01-01'))

admissions['admittime'] = pd.to_datetime(admissions['admittime'])

In [3]:
# 数据检查
for df in [diagnoses, admissions]:
    x = df[df['subject_id'].isin(patients['subject_id'])]['subject_id'].unique()
    x_ = df[~df['subject_id'].isin(patients['subject_id'])]['subject_id'].unique()
    print(x.shape, x_.shape)

(2417,) (0,)
(2417,) (0,)


In [4]:
# patients['dead_age'] = (patients['dod'] - patients['anchor_year']).dt.days / 365 + patients['anchor_age']

# patients

In [5]:
admissions = admissions.groupby('subject_id').first().reset_index()

for col in ['admittime', 'dischtime']:
    admissions[col] = pd.to_datetime(admissions[col])

In [6]:
data = patients[['subject_id', 'gender', 'anchor_age', 'anchor_year', 'dod']].merge(admissions[[
    'subject_id', 'admittime', 'insurance', 'language', 'marital_status', 'race']], on='subject_id', how='left')

In [7]:
diagnoses.shape

(122140, 5)

In [8]:
diagnoses = pd.concat([diagnoses['subject_id'], pd.get_dummies(diagnoses['icd_code'])], axis=1)

In [9]:
diagnoses = diagnoses.groupby('subject_id').any().reset_index()

In [10]:
diagnoses.loc[:, ~diagnoses.columns.isin(['subject_id'])] = diagnoses.loc[:, ~diagnoses.columns.isin(['subject_id'])].astype(int)

C:\Users\sitdo\AppData\Local\Temp\ipykernel_5076\3750952717.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  diagnoses.loc[:, ~diagnoses.columns.isin(['subject_id'])] = diagnoses.loc[:, ~diagnoses.columns.isin(['subject_id'])].astype(int)
C:\Users\sitdo\AppData\Local\Temp\ipykernel_5076\3750952717.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  diagnoses.loc[:, ~diagnoses.columns.isin(['subject_id'])] = diagnoses.loc[:, ~diagnoses.columns.isin(['subject_id'])].astype(int)
C:\Users\sitdo\AppData\Local\Temp\ipykernel_5076\3750952717.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a futur

In [11]:
diagnoses.loc[:, ~diagnoses.columns.isin(['subject_id'])].sum(axis=0).sort_values(ascending=False)

5559     1065
5569      863
4019      810
53081     678
2859      523
         ... 
27411       1
F430        1
F4312       1
F4325       1
0029        1
Length: 6546, dtype: int64

In [103]:
exclude_codes = [ '5550', '5551', '5552', '5559', '5560', '5561', '5562', '5563', '5564', '5565', '5566', '5568', '5569']

diagnoses.drop(exclude_codes, axis=1, inplace=True)

In [72]:
diagnoses.shape

(2417, 6534)

In [73]:
for col in data.columns:
    print(col)
    print(data[col].unique().shape)

subject_id
(2417,)
gender
(2,)
anchor_age
(73,)
anchor_year
(96,)
dod
(379,)
admittime
(2417,)
insurance
(3,)
language
(2,)
marital_status
(5,)
race
(28,)


In [82]:
def parse_race(race):
    if 'WHITE' in race:
        return 'WHITE'
    elif 'BLACK' in race:
        return 'BLACK'
    elif 'HISPANIC' in race or 'LATINO' in race:
        return 'HISPANIC/LATINO'
    elif 'ASIAN' in race:
        return 'ASIAN'
    else:
        return 'OTHER'

data['race'] = data['race'].apply(parse_race)

In [83]:
dummy_cols = ['gender', 'insurance', 'language', 'marital_status', 'race']

data = pd.concat([data.loc[:, ~data.columns.isin(dummy_cols)], pd.get_dummies(data[dummy_cols])], axis=1)

In [89]:
data['first_age'] = (data['admittime'] - data['anchor_year']).dt.days / 365 + data['anchor_age']

In [92]:
data['dod'] = data['dod'].isna().astype(int)

In [94]:
exclude_cols = ['anchor_age', 'anchor_year', 'admittime']

data = data.loc[:, ~data.columns.isin(exclude_cols)]

In [96]:
exclude_cols = ['subject_id', 'dod']
data.loc[:, ~data.columns.isin(exclude_cols)] = data.loc[:, ~data.columns.isin(exclude_cols)].astype(int)

In [105]:
data_fucked = data.merge(diagnoses, on='subject_id')

In [107]:
data_fucked.to_csv('../data/data.csv')